In [1]:
#Criado por: Vinícius de Almeida Nery Ferreira (ECO - UnB)

#######################################################################################################################
###COMO USAR AS FUNÇÕES EM UM NOTEBOOK
##Antes, copie e cole todos os imports e definições daqui na primeira célula do notebook e pressione Shift + Enter
##Para coletar os dados do arquivo "carros.dta" (só funciona com arquivos .dta):
#coletar_dados("carros")

#######################################################################################################################

#%%
##Importando os pacotes e módulos necessários
import pandas as pd
import numpy as np
import math

#Para Regressão Linear Simples e Teste F
from scipy import stats
#Para Regressão Linear Múltipla (OLS, GLS e WLS) e Testes Estatísticos
import statsmodels.api as sm
import econtools
import econtools.metrics as mt

#Pacotes para gráficos (caso precise)
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

#Pacotes para fazer a coleta dos dados armazenados no mesmo diretório e outros pacotes gerais
import os
import pathlib
import glob
from IPython.display import clear_output

##Criando as Funções
def coletar_dados(nome = ""):
    '''
    Função que le os arquivos do Stata (.dta) - NÃO COLOQUE A EXTENSÃO NA HORA DE NOMEAR O "NOME"!
    O arquivo deve estar na mesma pasta do arquivo de Python ou do notebook do jupyter.
    Deixe em branco para ler o arquivo mais recentemente adicionado à pasta.
    '''

    global df

    #Pegando qual a pasta do arquivo que está sendo usado pra programar
    caminho = pathlib.Path().absolute()

    #checando se o nome foi inserido ou não; caso não, pegar o arquivo .dta mais recente
    if nome == "":
        arquivo = max(glob.glob(f"{str(caminho)}/*.dta"), key=os.path.getctime)
        df = pd.read_stata(arquivo)
        print(f"{arquivo}.dta foi lido com sucesso!")
        return df
    else:
        try:
            arquivo = f"{str(caminho)}/{str(nome)}.dta"
            df = pd.read_stata(arquivo)
            print(f"{nome}.dta foi lido com sucesso!")
            return df
        except: #caso não tenha sido encontrado o arquivo com o nome inserido
            print('''
            Não foi possível achar o arquivo :(\n
            Verifique se seu nome está correto (sem a extensão) e se ele está no mesmo diretório do programa!
            ''')

def Regressão_Simples(Lista_x, Lista_y):
    '''
    Função que calcula as estatísticas de uma regressão simples

    Lista_x: lista com os valores de x;
    Lista_y: lista com os valores de y;
    '''

    global Lista_ychapeu_simples, Resíduos_simples
    
    #calculando o número de observações e as médias
    Número_de_Observações = len(Lista_x)
    Média_x = sum(Lista_x)/Número_de_Observações
    Média_y = sum(Lista_y)/Número_de_Observações

    #Calculando os coeficientes do modelo
    B1, B0, R, valor_P, DP = stats.linregress(Lista_x, Lista_y)
    #Gerando os valores previstos
    Lista_ychapeu_simples = [round(B0 + B1 * i,3) for i in Lista_x]
    #Calculando os Resíduos
    Resíduos_simples = [(j - k) for j,k in zip(Lista_y, Lista_ychapeu_simples)]
    
    #Calculando R-quadrados e a Soma dos Quadrados das Variáveis
    R_quadrado = round(R**2,5)
    SQTx = sum([(i - Média_x)**2 for i in Lista_x])
    SQTy = sum([(i - Média_y)**2 for i in Lista_y])
    SQEy = sum([(i - Média_y)**2 for i in Lista_ychapeu_simples])
    SQR = sum([i**2 for i in Resíduos_simples]) 

    #Calculando a Variância da Regressão e dos  Coeficientes
    VarianciaReg = SQR/(Número_de_Observações - 2)
    EPR = math.sqrt(VarianciaReg)
    VarB1 = VarianciaReg/SQTx
    VarB0 = (VarianciaReg * sum([i**2 for i in Lista_x]))/(Número_de_Observações * SQTx)
    
    #Calculando da estatistica t com intervalo de confiança de 95%  (p/ gerar os intervalos de confiança dos estimadores)
    Estatistica_t_Critica = stats.t.ppf(0.95, Número_de_Observações - 2)

    #Calculando os estimadores do limite inferior e superior
    B1_inferior = B1 - math.sqrt(VarB1) * Estatistica_t_Critica
    B1_superior = B1 + math.sqrt(VarB1) * Estatistica_t_Critica
    B0_inferior = B0 - math.sqrt(VarB0) * Estatistica_t_Critica
    B0_superior = B0 + math.sqrt(VarB0) * Estatistica_t_Critica
    
    #Gerando o Relatório
    Relatório = f'''
    Número de Observações = {Número_de_Observações}\n
    B0 = {round(B0,5)}\t B1 = {round(B1,5)}\t R-quadrado = {R_quadrado}\n
    Estimador da Variância = {round(VarianciaReg,5)}\t Erro Padrão da Regressão = {round(EPR,5)}\n
    Variância de B1 = {round(VarB1,5)}\t Variância de B0 = {round(VarB0,5)}\n
    Intervalo de Confiança de 95% para B1 (Inferior; B1; Superior): {round(B1_inferior,4)}; {round(B1,4)}; {round(B1_superior,4)}\n
    Intervalo de Confiança de 95% para B0 (Inferior; B0; Superior): {round(B0_inferior,4)}; {round(B0,4)}; {round(B0_superior,4)}\n
    Para ver os valores previstos, basta chamar a variável 'Lista_ychapeu_simples'\n
    Para ver os resíduos, chame a variável 'Resíduos_simples'
    '''
    print (Relatório)
    
    ##Criando um gráfico
    #Deixando o estilo bonitinho
    sns.set_style(style="white")

    #Criando o objeto gráfico
    Grafico = sns.regplot(x = Lista_x, y = Lista_y, scatter_kws={"color": "black"}, line_kws={"color": "red"})
    Grafico.set_title("Resultado da Regressão",fontsize = 11)
    plt.show()

def Regressão_Múltipla(x, y, constante = "S", robusta = "N"):
    '''
    Função que calcula uma regressão múltipla, sendo, por default, computada com um intercepto e com erros padrões não robustos.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns.
    '''

    global Resultado, Lista_ychapeu, Resíduos, SQR, EPR

    #adicionando uma constante ao modelo de Ordinary Least Squares (OLS)
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x

    #Criando o Modelo levando em conta a opção de ser uma regressão robusta p/ heteroscedasticidade ou não
    Modelo = sm.OLS(y,X)

    if robusta == "N":
        Resultado = Modelo.fit()
    else:
        Resultado = Modelo.fit(cov_type = 'HC1', use_t = True)
    
    Lista_ychapeu = Resultado.predict()
    Resíduos = y - Lista_ychapeu

    #Calculando o Erro Padrão da Regressão (EPR)
    SQR =sum([i**2 for i in Resíduos])
    Número_de_Observações = len(y)
    GL = Número_de_Observações - len(Resultado.params)
    VarianciaReg = SQR/GL
    EPR = math.sqrt(VarianciaReg)
    
    ##Printando o Resultado
    #print('Parâmetros:\n', Resultado.params) #O primeiro resultado equivale ao intercepto
    #print('\nDesvios Padrões:\n', Resultado.bse)
    #print('Valores Previstos: ', Resultado.predict())
    #print('\nR2:', Resultado.rsquared)
    print(f"O erro padrão da regressão é {round(EPR,5)} e a SQR é {round(SQR,5)}\n")
    print(Resultado.summary())

    print("\nPara ver os valores previstos ou os resídudos, basta chamar 'Lista_ychapeu' e 'Resíduos'.")
    print("Os resultados do modelo podem ser obtidos através de métodos usando a variável 'Resultado'.")
    print("""
    Valores de condição maiores que 20 indicam problemas de multicolinearidade
    Para ver como achar esse número, entre em https://www.statsmodels.org/dev/examples/notebooks/generated/ols.html"""
    )

def Regressão_MQP(x, y, pesos, constante = "S", robusta = "N"):
    '''
    Função que calcula uma regressão múltipla usando mínimos quadrados ponderados, ou seja,
    recomendada quando o erro é heteroscedástico E se sabe a função da constante. Ela é, por default, computada com um intercepto e com erros padrões não robustos.
    multiplicativa da variância do erro.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    pesos: 1/h, sendo h a constante multiplicativa da variância do erro;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns.
    '''

    global Resultado, Lista_ychapeu, Resíduos, SQR, EPR

    #adicionando uma constante ao modelo de Ordinary Least Squares(OLS)
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x
    #Criando o Modelo levando em conta a opção de ser uma regressão robusta p/ heteroscedasticidade ou não
    Modelo = sm.WLS(y,X, weights = pesos)
    if robusta == "N":
        Resultado = Modelo.fit()
    else:
        Resultado = Modelo.fit(cov_type = 'HC1', use_t = True)
    
    Lista_ychapeu = Resultado.predict()
    Resíduos = y - Lista_ychapeu

    #Calculando o Erro Padrão da Regressão (EPR)
    SQR =sum([i**2 for i in Resíduos])
    Número_de_Observações = len(y)
    GL = Número_de_Observações - len(Resultado.params)
    VarianciaReg = SQR/GL
    EPR = math.sqrt(VarianciaReg)
    
    ##Printando o Resultado
    print(f"O erro padrão da regressão é {round(EPR,5)} e a SQR é {round(SQR,5)}\n")
    print(Resultado.summary())

    print("\nPara ver os valores previstos ou os resídudos, basta chamar 'Lista_ychapeu' e 'Resíduos'.")
    print("Os resultados do modelo podem ser obtidos através de métodos usando a variável 'Resultado'.")
    print("""
    Valores de condição maiores que 20 indicam problemas de multicolinearidade
    Para ver como achar esse número, entre em https://www.statsmodels.org/dev/examples/notebooks/generated/ols.html"""
    )
    
def Regressão_MQGF(x, y, constante = "S", robusta = "N"):
    '''
    Função que calcula uma regressão múltipla usando mínimos quadrados generalizados factíveis, ou seja,
    recomendada quando o erro é heteroscedástico E NÃO se sabe a função da constante multiplicativa da variância do erro, sendo os pesos estimados
    regridindo o log dos quadrados dos resíduos sobre as variáveis explicativas. Os estimadores MQP são gerados com o peso estimado.
    Ela é, por default, computada com um intercepto e com erros padrões não robustos.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns.
    '''

    global Resultado, Lista_ychapeu, Resíduos, SQR, EPR

    #Regredindo os valores normalmente a fim de pegar os resíduos
    Regressão_Múltipla(x,y, constante, robusta)
    clear_output()

    #Coletando o log dos quadrados dos resíduos
    Log_Res_Quad = np.log(Resíduos**2)

    #Regredindo Log_Res_Quad sobre as variáveis explicativas
    Regressão_Múltipla(x,Log_Res_Quad, constante, robusta)
    clear_output()

    #Estimando os pesos
    Pesos = np.exp(Lista_ychapeu)

    #Fazendo uma Regressão MQP
    Regressão_MQP(x,y, 1/Pesos, constante, robusta)

def Teste_LM(x, y, Restrições, Nivel_de_Significância = 0.05):
    '''
    Função que calcula um teste LM e dá o resultado teste de hipótese para o caso de todas as restrições serem conjuntamente estatisticamente não-significantes.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    Restrições: lista ou array com os valores a serem tirados do modelo restrito;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''

    ##Definindo as variáveis de cada modelo
    ModeloIrrestrito = list(x)
    ModeloRestrito = []
    Restrições = list(Restrições)

    Numero_de_Observações = len(y)
    GL_ir = Numero_de_Observações - (len(ModeloIrrestrito) + 1)
    GL_r = len(Restrições)

    for i in ModeloIrrestrito:
        if i not in Restrições:
            ModeloRestrito.append(i)
    
    #Fazendo a regressão do modelo restrito e armazenando os resíduos
    Regressão_Múltipla(df[ModeloRestrito], y)
    Resíduos_r = Resíduos

    #Fazendo a regressão dos resíduos sobre as variáveis independentes e armazenando o R2
    Regressão_Múltipla(x, Resíduos_r)
    Ru = Resultado.rsquared

    #Calculando a estatística LM
    LM = Numero_de_Observações*Ru

    #Calculando o p-valor
    ##Calculando o P-valor de F
    P_valor = stats.chi2.sf(LM,GL_r)

    #Limpando a tela
    clear_output()

    #Printando o resultado
    if Nivel_de_Significância > P_valor:
        print(f"O valor de LM é {round(LM,3)} e seu p-valor é {round(P_valor,7)}. Portanto, rejeita-se Ho a um nível de significância de {Nivel_de_Significância*100}%.")
    else:
        print(f"O valor de LM é {round(LM,3)} e seu p-valor é {round(P_valor,7)}. Portanto, não se rejeita Ho a um nível de significância de {Nivel_de_Significância*100}%.")


def Teste_F(x, y, Restrições, Nivel_de_Significância = 0.05):
    '''
    Função que calcula um teste F e dá o resultado teste de hipótese para o caso de todas as restrições serem conjuntamente estatisticamente não-significantes.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    Restrições: lista ou array com os valores a serem tirados do modelo restrito;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''

    ##Definindo as variáveis de cada modelo
    #para testar igualdade dos coeficientes, F2, p_valueF2 = results.Ftest(['ACT', 'skipped'], equal=True)
    ModeloIrrestrito = list(x)
    ModeloRestrito = []
    Restrições = list(Restrições)

    Numero_de_Observações = len(y)
    GL_ir = Numero_de_Observações - (len(ModeloIrrestrito) + 1)
    GL_r = len(Restrições)

    for i in ModeloIrrestrito:
        if i not in Restrições:
            ModeloRestrito.append(i)

    ##Fazendo as regressões de cada modelo
    Regressão_Múltipla(x, y)
    SQR_ir = SQR
    VarianciaReg_ir = EPR**2

    Regressão_Múltipla(df[ModeloRestrito], y)
    SQR_r = SQR

    #Limpando a tela
    clear_output()
    
    ##Calculando F
    F = (SQR_r - SQR_ir)/(len(Restrições)*VarianciaReg_ir)

    ##Calculando o P-valor de F
    P_valor = stats.f.sf(F,GL_r,GL_ir)

    if Nivel_de_Significância > P_valor:
        print(f"O valor de F é {round(F,3)} e seu p-valor é {round(P_valor,7)}. Portanto, rejeita-se Ho a um nível de significância de {Nivel_de_Significância*100}%.")
    else:
        print(f"O valor de F é {round(F,3)} e seu p-valor é {round(P_valor,7)}. Portanto, não se rejeita Ho a um nível de significância de {Nivel_de_Significância*100}%.")

def Teste_t_Dois_Coeficientes_Iguais(x, y, Coeficientes_Testados_para_serem_iguais, Nivel_de_Significância = 0.05):
    '''
    Função que executa um teste t para verificar se dois coeficientes são iguais.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    Coeficientes_Testados_para_serem_iguais: array com os valores dos coeficientes que querem ser testados;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''
    
    ##Fazendo a regressão do modelo irrestrito
    Regressão_Múltipla(x, y)
    clear_output()

    #Fazendo o objeto de lista que será usado no teste
    Teste =[0]
    Num_de_Variaveis = 1

    for i in list(x):
        if i not in list(Coeficientes_Testados_para_serem_iguais):
            Teste.append(0)
        elif (Num_de_Variaveis % 2 == 0):
            Teste.append(-1)
        else:
            Teste.append(1)
            Num_de_Variaveis += 1

    Teste_t = Resultado.t_test(Teste)
    print(f"A estatística do teste é {np.around(Teste_t.tvalue[0],3)}, o que resulta em um p-valor de {np.around(Teste_t.pvalue[0],6)}")

def Teste_Heteroscedasticidade_BP(x, y, constante = "S", Nivel_de_Significância = 0.05, Estatística = "LM"):
    '''
    Função que executa o teste de Breusch-Pagan para a heteroscedasticidade.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    Estatística = LM ou F
    '''

    #Fazendo a regressão e limpando a tela
    Regressão_Múltipla(x,y,constante)
    clear_output()

    #Calculando o quadrado dos resíduos
    Res_Quad = Resíduos**2

    #Realizando o teste F ou LM de Res_Quad sobre as variaveis dependentes para ver se há correlação
    if Estatística == "LM":
        Teste_LM(x, Res_Quad, x, Nivel_de_Significância)
        print("Ho: O erro é homoscedástico")
    else:
        Teste_F(x, Res_Quad, x, Nivel_de_Significância)
        print("Ho: O erro é homoscedástico")

def Teste_Heteroscedasticidade_White(x, y, constante = "S", Nivel_de_Significância = 0.05, Estatística = "LM"):
    '''
    Função que executa o teste de White (modificado por Wooldridge) para a heteroscedasticidade.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    Estatística: LM ou F
    '''

    #Fazendo a regressão e limpando a tela
    Regressão_Múltipla(x,y,constante)
    clear_output()

    #Calculando o quadrado dos resíduos
    Res_Quad = Resultado.resid**2

    #Calculando o quadrado dos valores previstos
    Previstos = Lista_ychapeu
    Previstos2 = Previstos**2

    #Criando um dataframe pra armazenar esses valores
    dfy_y2 = pd.DataFrame({"y":Previstos,"y2":Previstos2})
    y_y2 = dfy_y2[['y','y2']]

    #Realizando o teste F ou LM de Res_Quad sobre y e y^2
    if Estatística == "LM":
        Teste_LM(y_y2, Res_Quad, y_y2, Nivel_de_Significância)
        print("Ho: O erro é homoscedástico")
    else:
        Teste_F(y_y2, Res_Quad, y_y2, Nivel_de_Significância)
        print("Ho: O erro é homoscedástico")

def RESET(x, y, constante = "S", robusta = "N", Nivel_de_Significância = 0.05):
    '''
    Função que executa um teste RESET para verificar a adequação das formas funcionais.
    Ho: o modelo está bem especificado.

    x: lista ou array com os valores das variáveis independentes;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''
    #Fazendo uma regressão múltipla e limpando a tela
    Regressão_Múltipla(x, y, constante)
    clear_output()

    #Verificando o tipo da covariância selecionada
    if robusta == "N":
        tipo = 'nonrobust'
    else:
        tipo = 'HC1'

    Teste = sm.stats.diagnostic.linear_reset(Resultado, power = 2, use_f = False, cov_type = tipo)
    
    if Teste.pvalue < Nivel_de_Significância:
        print(f"""
        O p-valor do teste foi de {np.around(Teste.pvalue,6)}, menor que o nível de significância de {Nivel_de_Significância*100}%.\n
        Assim, rejeita-se Ho (o modelo está MAL especificado)."""
        )
    else:
        print(f"""
        O p-valor do teste foi de {np.around(Teste.pvalue,6)}, maior que o nível de significância de {Nivel_de_Significância*100}%.\n
        Assim, não se rejeita Ho (o modelo NÃO está MAL especificado)"""
        )

def Teste_J_Davidson_MacKinnon(x1,x2, y, constante = "S", robusta = "N", Nivel_de_Significância = 0.05):
    '''
    Função que executa um teste J para verificar qual o modelo mais adequado (dentre os dois colocados).
    Ho: o modelo 1 é preferível (ver o p-valor do último coeficiente).

    x1: lista ou array com os valores das variáveis independentes do primeiro modelo;
    x2: lista ou array com os valores das variáveis independentes do segundo modelo;
    y: lista ou array com os valores da variável dependente;
    constante: "S" para regressão com intercepto e qualquer outro valor para sem intercepto. Caso em branco, a regressão é computada com intercepto;
    robusta: "N" para regressão com erros-padrão tradicionais e qualquer outro valor para erros-padrões robustos. Caso em branco, a regressão é computada com erros-padrão comuns;
    Nivel_de_Significância: nível de significância do teste. Caso branco, o nível de significancia padrão é de 5%.
    '''
    
    #Fazendo a regressão do segundo modelo
    Regressão_Múltipla(x2, y, constante, robusta)
    clear_output()

    #Criando um novo dataframe e adicionando os valores previstos do modelo 2 à x
    Valores_Previstos_2 = pd.DataFrame({'Previsão M1':Lista_ychapeu})
    x = pd.concat([x1, Valores_Previstos_2], axis=1, sort=False)

    #Fazendo a regressão do primeiro modelo sobre x
    Regressão_Múltipla(x, y, constante, robusta)
    clear_output()

    #Pegando o p-valor do teste
    P_valor = Resultado.pvalues[-1]

    if P_valor < Nivel_de_Significância:
        print(f"""
        O p-valor do teste foi de {np.around(P_valor,6)}, menor que o nível de significância de {Nivel_de_Significância*100}%.\n
        Assim, rejeita-se Ho (ou seja, o modelo 2 ({list(x2)}) é mais bem especificado)."""
        )
    else:
        print(f"""
        O p-valor do teste foi de {np.around(P_valor,6)}, menor que o nível de significância de {Nivel_de_Significância*100}%.\n
        Assim, não se rejeita Ho (ou seja, o modelo 1 ({list(x1)}) é mais bem especificado)."""
        )


# Questão 2

In [2]:
coletar_dados("dadosmunicipios")

dadosmunicipios.dta foi lido com sucesso!


,escol,pib,pop,desig,pobreza,poprural,analf,rural,pibpc,ne,se,su,co,no,escolne,escolse,escolsu,escolco,escolno
0,3.7,14649.976562,10208.0,0.632,51.840000,3450.0,20.160000,0.337970,1.435147,0,0,0,0,1,0.0,0.0,0.0,0.0,3.7
1,3.0,11227.611328,6730.0,0.667,58.507999,1459.0,20.412001,0.216790,1.668293,0,0,0,0,1,0.0,0.0,0.0,0.0,3.0
2,2.5,6927.590332,7121.0,0.542,68.810997,3638.0,25.604000,0.510883,0.972840,0,0,0,0,1,0.0,0.0,0.0,0.0,2.5
3,1.8,44991.011719,51090.0,0.536,81.734001,34612.0,39.916000,0.677471,0.880623,0,0,0,0,1,0.0,0.0,0.0,0.0,1.8
4,NaN,50836.183594,11463.0,0.653,43.911999,8463.0,22.796000,0.738288,4.434806,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5503,1.4,20115.060547,18120.0,0.582,73.195000,10169.0,42.999001,0.561203,1.110103,1,0,0,0,0,1.4,0.0,0.0,0.0,0.0
5504,2.6,21157.789062,31819.0,0.523,76.162003,15059.0,36.916000,0.473271,0.664942,1,0,0,0,0,2.6,0.0,0.0,0.0,0.0
5505,4.0,20469.505859,18945.0,0.526,54.722000,NaN,18.091999,NaN,1.080470,1,0,0,0,0,4.0,0.0,0.0,0.0,0.0
5506,NaN,NaN,4883.0,0.426,76.162003,3509.0,59.862000,0.718616,NaN,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN


In [3]:
#Limpando o dataset
df.dropna(subset = ['escol'], axis = 0, inplace = True)
#resetando o index
df.reset_index(drop = True, inplace = True)

len(df)

4491

In [4]:
##Fazendo a regressão por MQO e sem inferência robusta
df['escol_no'] = df['escol']*df['no']
df['escol_ne'] = df['escol']*df['ne']
df['escol_su'] = df['escol']*df['su']
df['escol_co'] = df['escol']*df['co']

Regressão_Múltipla(df[['no','ne','su','co','pibpc','escol','escol_no','escol_ne','escol_su','escol_co']],df['pobreza'], "S", "N")

O erro padrão da regressão é 8.74366 e a SQR é 342503.15107

                            OLS Regression Results                            
Dep. Variable:                pobreza   R-squared:                       0.852
Model:                            OLS   Adj. R-squared:                  0.852
Method:                 Least Squares   F-statistic:                     2588.
Date:                Wed, 18 Nov 2020   Prob (F-statistic):               0.00
Time:                        11:26:37   Log-Likelihood:                -16105.
No. Observations:                4491   AIC:                         3.223e+04
Df Residuals:                    4480   BIC:                         3.230e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

In [5]:
##Fazendo a regressão por MQO e COM inferência robusta
Regressão_Múltipla(df[['no','ne','su','co','pibpc','escol','escol_no','escol_ne','escol_su','escol_co']],df['pobreza'], "S", "S")

O erro padrão da regressão é 8.74366 e a SQR é 342503.15107

                            OLS Regression Results                            
Dep. Variable:                pobreza   R-squared:                       0.852
Model:                            OLS   Adj. R-squared:                  0.852
Method:                 Least Squares   F-statistic:                     3081.
Date:                Wed, 18 Nov 2020   Prob (F-statistic):               0.00
Time:                        11:26:37   Log-Likelihood:                -16105.
No. Observations:                4491   AIC:                         3.223e+04
Df Residuals:                    4480   BIC:                         3.230e+04
Df Model:                          10                                         
Covariance Type:                  HC1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

In [6]:
##Calculando a resposta do item 2C

#Chaves para o dicionário
Chaves = ['INT','NO','NE','SU','CO','PIBPC','ESCOL','ESCOL_NO','ESCOL_NE','ESCOL_SU','ESCOL_CO']

#Vetor de dados para um municipio do SE, sendo o primeiro um valor para o intercepto
SE = [1, 0,0,0,0,3.1,3.14,0,0,0,0]
SE_dict = dict(zip(Chaves,SE))

#Vetor de dados para um municipio do NE, sendo o primeiro um valor para o intercepto
NE = [1, 0,1,0,0,3.1,3.14,0,3.14,0,0]
NE_dict = dict(zip(Chaves,NE))

#Printando os valores pra conferir
print(f"SE: {SE_dict}\n NE: {NE_dict}")

SE: {'INT': 1, 'NO': 0, 'NE': 0, 'SU': 0, 'CO': 0, 'PIBPC': 3.1, 'ESCOL': 3.14, 'ESCOL_NO': 0, 'ESCOL_NE': 0, 'ESCOL_SU': 0, 'ESCOL_CO': 0}
 NE: {'INT': 1, 'NO': 0, 'NE': 1, 'SU': 0, 'CO': 0, 'PIBPC': 3.1, 'ESCOL': 3.14, 'ESCOL_NO': 0, 'ESCOL_NE': 3.14, 'ESCOL_SU': 0, 'ESCOL_CO': 0}


In [7]:
##Fazendo as estimações
#np.dot é uma função do numpy que faz um produto interno entre vetores
print(f"""
    Estimação de SE: {np.around(np.dot(list(Resultado.params), SE),5)}\n
    Estimação de NE: {np.around(np.dot(list(Resultado.params), NE),5)}\n
    Diferença (SE - NE): {np.around(np.dot(list(Resultado.params), SE) - np.dot(list(Resultado.params), NE),5)}"""
)



    Estimação de SE: 39.02457

    Estimação de NE: 57.78286

    Diferença (SE - NE): -18.75829


In [8]:
## 2d
#Os efeitos_educ foram calculados derivando a regressão com relação a educ

Regiões = ['SE','NO','SU','CO','NE']
Efeitos_Educ = [-12.41,-10.31, -9.76, -9.51, -8.83]
Efeitos_Educ_Regiões = dict(zip(Regiões,Efeitos_Educ))

print(Efeitos_Educ_Regiões)

{'SE': -12.41, 'NO': -10.31, 'SU': -9.76, 'CO': -9.51, 'NE': -8.83}


In [9]:
#Analisando as desigualdades do SE e do NE, bem como sua escolaridade média e pib per capita médio
df_ne = df.loc[df['ne'] == 1]
df_se = df.loc[(df['ne'] == 0) & (df['no'] == 0) & (df['su'] == 0) & (df['co'] == 0)]

In [10]:
df_ne.describe()

,escol,pib,pop,desig,pobreza,poprural,analf,rural,pibpc,ne,...,no,escolne,escolse,escolsu,escolco,escolno,escol_no,escol_ne,escol_su,escol_co
count,1509.000000,1.509000e+03,1.509000e+03,1509.000000,1509.000000,1498.000000,1509.000000,1498.000000,1509.000000,1509.0,...,1509.0,1509.000000,1509.0,1509.0,1509.0,1509.0,1509.0,1509.000000,1509.0,1509.0
mean,2.006362,7.961412e+04,3.033734e+04,0.537396,68.650024,9060.873832,42.217346,0.479568,1.261711,1.0,...,0.0,2.006362,0.0,0.0,0.0,0.0,0.0,2.006362,0.0,0.0
std,0.825272,6.447999e+05,1.059157e+05,0.108051,9.838807,7780.458770,9.221081,0.193751,1.497261,0.0,...,0.0,0.825272,0.0,0.0,0.0,0.0,0.0,0.825272,0.0,0.0
min,0.400000,3.572565e+02,1.308000e+03,0.185000,2.741000,34.000000,6.686000,0.000411,0.087306,1.0,...,0.0,0.400000,0.0,0.0,0.0,0.0,0.0,0.400000,0.0,0.0
25%,1.500000,7.455299e+03,8.673000e+03,0.463000,63.771999,3818.000000,37.202000,0.339632,0.743362,1.0,...,0.0,1.500000,0.0,0.0,0.0,0.0,0.0,1.500000,0.0,0.0
50%,1.800000,1.404258e+04,1.501400e+04,0.524000,70.214996,7174.500000,43.060001,0.492068,0.962786,1.0,...,0.0,1.800000,0.0,0.0,0.0,0.0,0.0,1.800000,0.0,0.0
75%,2.300000,2.764470e+04,2.602200e+04,0.593000,75.167999,11907.500000,48.650002,0.624977,1.291097,1.0,...,0.0,2.300000,0.0,0.0,0.0,0.0,0.0,2.300000,0.0,0.0
max,6.900000,1.578945e+07,2.443107e+06,1.174000,89.331001,80139.000000,63.709000,0.984407,32.249939,1.0,...,0.0,6.900000,0.0,0.0,0.0,0.0,0.0,6.900000,0.0,0.0


In [11]:
df_se.describe()

,escol,pib,pop,desig,pobreza,poprural,analf,rural,pibpc,ne,...,no,escolne,escolse,escolsu,escolco,escolno,escol_no,escol_ne,escol_su,escol_co
count,1432.000000,1.432000e+03,1.432000e+03,1432.000000,1432.000000,1400.000000,1432.000000,1400.000000,1432.000000,1432.0,...,1432.0,1432.0,1432.000000,1432.0,1432.0,1432.0,1432.0,1432.0,1432.0,1432.0
mean,3.804609,3.832703e+05,4.881194e+04,0.500548,30.804546,4428.149286,17.539379,0.283394,3.966513,0.0,...,0.0,0.0,3.804609,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,1.020999,4.373650e+06,3.316661e+05,0.097710,16.503420,17041.200255,8.522155,0.194681,3.182485,0.0,...,0.0,0.0,1.020999,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.800000,2.340898e+03,7.950000e+02,0.290000,2.892000,7.000000,3.105000,0.000277,0.500990,0.0,...,0.0,0.0,0.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,3.200000,1.405737e+04,5.355500e+03,0.433000,18.491000,1277.500000,11.881250,0.126065,2.158315,0.0,...,0.0,0.0,3.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,3.800000,3.338457e+04,1.150850e+04,0.489500,26.096000,2659.000000,15.410000,0.247807,3.242882,0.0,...,0.0,0.0,3.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,4.400000,1.074550e+05,2.671975e+04,0.554000,39.467251,5079.250000,20.617500,0.416419,4.759246,0.0,...,0.0,0.0,4.400000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,8.800000,1.412110e+08,1.043425e+07,1.062000,78.889999,621065.000000,56.660000,0.856077,50.640675,0.0,...,0.0,0.0,8.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
Teste_Heteroscedasticidade_White(df[['no','ne','su','co','pibpc','escol','escol_no','escol_ne','escol_su','escol_co']],df['pobreza'])

O valor de LM é 154.634 e seu p-valor é 0.0. Portanto, rejeita-se Ho a um nível de significância de 5.0%.
Ho: O erro é homoscedástico


In [13]:
Teste_Heteroscedasticidade_BP(df[['no','ne','su','co','pibpc','escol','escol_no','escol_ne','escol_su','escol_co']],df['pobreza'])

O valor de LM é 381.697 e seu p-valor é 0.0. Portanto, rejeita-se Ho a um nível de significância de 5.0%.
Ho: O erro é homoscedástico


# Questão 3

In [14]:
coletar_dados("xydata")

xydata.dta foi lido com sucesso!


,y,x
0,459.600006,85.000000
1,2048.899902,335.000000
2,1587.199951,425.100006
3,1540.099976,205.399994
4,279.700012,24.299999
5,794.200012,55.200001
6,38.599998,493.700012
7,1600.699951,362.899994
8,923.299988,58.500000
9,5905.299805,490.100006


In [15]:
#Fazendo uma regressão MQO
Regressão_Múltipla(df['x'],df['y'],"S","N")

O erro padrão da regressão é 1524.0169 e a SQR é 18581019.99158

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.264
Model:                            OLS   Adj. R-squared:                  0.173
Method:                 Least Squares   F-statistic:                     2.877
Date:                Wed, 18 Nov 2020   Prob (F-statistic):              0.128
Time:                        11:26:38   Log-Likelihood:                -86.365
No. Observations:                  10   AIC:                             176.7
Df Residuals:                       8   BIC:                             177.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [16]:
#Testando para heteroscedasticidade
Teste_Heteroscedasticidade_White(df['x'],df['y'])

O valor de LM é 7.627 e seu p-valor é 0.0220757. Portanto, rejeita-se Ho a um nível de significância de 5.0%.
Ho: O erro é homoscedástico


In [17]:
#Usando erros padrões robustos
Regressão_Múltipla(df['x'],df['y'],"S","S")

O erro padrão da regressão é 1524.0169 e a SQR é 18581019.99158

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.264
Model:                            OLS   Adj. R-squared:                  0.173
Method:                 Least Squares   F-statistic:                     1.687
Date:                Wed, 18 Nov 2020   Prob (F-statistic):              0.230
Time:                        11:26:39   Log-Likelihood:                -86.365
No. Observations:                  10   AIC:                             176.7
Df Residuals:                       8   BIC:                             177.3
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [18]:
#Usando MQP
df['x2'] = df['x']**2
Regressão_MQP(df['x'],df['y'],1/df['x2'],"S","N")

O erro padrão da regressão é 1552.04793 e a SQR é 19270822.33721

                            WLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.529
Model:                            WLS   Adj. R-squared:                  0.470
Method:                 Least Squares   F-statistic:                     8.986
Date:                Wed, 18 Nov 2020   Prob (F-statistic):             0.0171
Time:                        11:26:39   Log-Likelihood:                -79.333
No. Observations:                  10   AIC:                             162.7
Df Residuals:                       8   BIC:                             163.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------